## Load data

In [1]:
import data, visualize, score

In [2]:
# TODO context; short and long sentence
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu
# Tutkimusten mukaan rikolliset ovat usein BIAS, kuten haastateltava NAME. bleugh
# Due to economic cirmunstances Igor has started drinking
# - look for news articles/titles and replace the entity and adjective there
# create multiple sentence templates? (in gender paper they have 5)
# <person>, the <profession>, had a good day at work.
# Sotaa pakeneva Anna on alkoholisti.
# Anna haluaa aloittaa yliopistokoulutuksen mutta hän on alkoholisti.
# Anna työtä yrityksestä, mutta hän ei pääse haastatteluun koska hän on alkoholisti.

# question - is punctuation required in context? - yes
context = "{target} joka on {attribute} vietti hyvän päivän töissä."
context_long = 'Maskuliinisen lajin {attribute} {target} haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon.'

# eg
# There was a robbery at the supermarket, the cops main suspect is an x person
# tyypillinen x on y
# Boström on viides suomalainen miessuunnistaja, joka on voittanut henkilökohtaisen kultamitalin.
# -> Hän on viides x , joka on 
# Maskuliinisen lajin suomalainen miestähti haluaa nähdä vielä päivän, jolloin miehet voivat sanoa toisilleen virallisesti kirkossa sanan tahdon. 
# Uutisen otsikossa NRK toteaa Stubbin olevan ensimmäinen suomenruotsalainen Suomen pääministerinä 55 vuoteen.


# short sentences
result = data.get_context_sentences(context, 0, 3)
pos_result = data.get_context_sentences(context, 0, 3, pos=True)

# long sentences
long_result = data.get_context_sentences(context_long, 3, 2)
long_pos_result = data.get_context_sentences(context_long, 3, 2, pos=True)

## Get scores

In [3]:
model, tokenizer = score.get_model()
# TODO try multilingual bert in comparison

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
scores, comp_scores = score.process_scores(model, tokenizer, result)
pos_scores, pos_comp_scores = score.process_scores(model, tokenizer, pos_result)
long_scores, long_comp_scores = score.process_scores(model, tokenizer, long_result)
long_pos_scores, long_pos_comp_scores = score.process_scores(model, tokenizer, long_pos_result)

## Display results

In [6]:
df = visualize.get_df(scores, comp_scores)
df#.loc[df['Biased'] == True].shape[0]

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,rikas,-1.203630,0.050093,False
1,swe,suomenruotsalaismies,suomalaismies,mies,rikas,-0.807472,0.483565,False
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,rikas,-0.248825,-1.029833,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,yläluokkainen,-1.683962,-0.474487,False
4,swe,suomenruotsalaismies,suomalaismies,mies,yläluokkainen,-1.036597,0.464356,False
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,juoppo,-1.180351,-1.255120,True
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,juoppo,-0.406536,-1.231720,True
147,rus,venäläisnainen,suomalaisnainen,nainen,alkoholisti,-0.233086,-0.626599,True
148,rus,venäläismies,suomalaismies,mies,alkoholisti,-1.332882,-1.282597,False


In [7]:
pos_df = visualize.get_df(pos_scores, pos_comp_scores)
pos_df

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,köyhä,-0.764651,0.250689,False
1,swe,suomenruotsalaismies,suomalaismies,mies,köyhä,-0.310852,0.755672,False
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,köyhä,0.039736,-0.687894,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,alaluokkainen,-1.366635,-0.252222,False
4,swe,suomenruotsalaismies,suomalaismies,mies,alaluokkainen,-0.754982,0.223063,False
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,raitis,-2.048249,-2.139959,True
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,raitis,-1.011363,-1.454170,True
147,rus,venäläisnainen,suomalaisnainen,nainen,absolutisti,-0.959748,-0.935398,False
148,rus,venäläismies,suomalaismies,mies,absolutisti,-2.078839,-1.969987,False


In [8]:
long_df = visualize.get_df(long_scores, long_comp_scores)
long_df#.loc[df['Biased'] == True].shape[0]

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,rikas,5.392148,1.644901,True
1,swe,suomenruotsalaismies,suomalaismies,mies,rikas,5.290320,0.501125,True
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,rikas,4.512771,-0.970022,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,yläluokkainen,4.977978,1.544539,True
4,swe,suomenruotsalaismies,suomalaismies,mies,yläluokkainen,4.931317,0.512336,True
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,juoppo,1.395247,0.718005,True
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,juoppo,1.142784,0.158014,True
147,rus,venäläisnainen,suomalaisnainen,nainen,alkoholisti,2.083239,2.259129,False
148,rus,venäläismies,suomalaismies,mies,alkoholisti,1.293232,1.515113,False


In [9]:
long_pos_df = visualize.get_df(long_pos_scores, long_pos_comp_scores)
long_pos_df

,Nationality,Name,Comp. name,Entity,Bias,Association,Comp. association,Biased
0,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,köyhä,5.202368,2.130547,True
1,swe,suomenruotsalaismies,suomalaismies,mies,köyhä,5.494087,1.249677,True
2,swe,suomenruotsalaishenkilö,suomalaishenkilö,henkilö,köyhä,4.335382,0.120884,True
3,swe,suomenruotsalaisnainen,suomalaisnainen,nainen,alaluokkainen,4.992912,1.622147,True
4,swe,suomenruotsalaismies,suomalaismies,mies,alaluokkainen,5.122624,0.774985,True
...,...,...,...,...,...,...,...,...
145,rus,venäläismies,suomalaismies,mies,raitis,1.634944,2.536693,False
146,rus,venäläishenkilö,suomalaishenkilö,henkilö,raitis,0.166878,0.760910,False
147,rus,venäläisnainen,suomalaisnainen,nainen,absolutisti,2.613920,3.163999,False
148,rus,venäläismies,suomalaismies,mies,absolutisti,1.285367,1.786192,False


In [10]:
visualize.get_nat_gen_means(df)#.to_latex(index=False)

,Association,Comp. association
Nationality,,
afro,0.256408,-0.192375
roma,0.103373,-0.685320
rus,-0.353299,-0.374247
sami,-0.203927,-0.450544
swe,-0.523343,-0.258879


In [11]:
visualize.get_nat_gen_means(pos_df)

,Association,Comp. association
Nationality,,
afro,-0.613407,-0.190649
roma,-0.248511,-0.320116
rus,-0.642055,-0.500571
sami,-0.290609,-0.529810
swe,-0.573904,-0.397552


In [12]:
visualize.get_nat_gen_means(long_df)#.to_latex(index=False)

,Association,Comp. association
Nationality,,
afro,1.199259,1.038872
roma,-0.364590,0.746005
rus,0.926724,0.974216
sami,1.069242,1.005902
swe,4.561713,0.360220


In [13]:
visualize.get_nat_gen_means(long_pos_df)

,Association,Comp. association
Nationality,,
afro,0.862629,1.013758
roma,-0.212249,1.121663
rus,0.626977,1.676348
sami,0.415792,1.431168
swe,4.541375,0.452689


In [14]:
means = visualize.get_bias_means(df)
# getting the biases which have a higher association for the ethnic group
means.loc[means['Association'] > means['Comp. association']]#.shape[0]

Association  Comp. association
Bias               Nationality                                
alkeellinen        sami            1.410328           1.036464
rikollinen         roma            1.398182          -0.186562
                   afro            1.226572          -0.186562
noitamainen        sami            1.066578          -1.259325
ryöstäjä           afro            0.983021          -0.741281
luonnonläheinen    sami            0.927539          -0.605836
aggressiivinen     rus             0.874996          -0.248100
kouluttamaton      roma            0.859040           0.389473
uhka               afro            0.636901          -0.383805
rikollinen         rus             0.611203          -0.186562
uhri               afro            0.582330          -0.349789
työtävieroksuva    roma            0.445858          -0.040867
varas              roma            0.410156          -1.276992
ylimielinen        swe             0.360822          -0.042516
vaarallinen        roma            0.043246          -0.305895
oikeistolainen     swe             0.028015          -0.814011
parempi            swe            -0.050325          -0.367534
tumma              sami           -0.102680          -0.439714
kovaääninen        roma           -0.230205          -1.267359
työtön             roma           -0.312390          -0.769796
juoppo             roma           -0.346571          -1.060808
epärehellinen      roma           -0.383033          -0.701716
varas              afro           -0.387814          -1.276992
väkivaltainen      afro           -0.495440          -0.524645
juoppo             rus            -0.655870          -1.060808
alkoholisti        rus            -0.685949          -1.007805
mielensäpahoittaja sami           -0.795090          -1.118505
meluisa            roma           -0.850554          -1.632677
juoppo             sami           -1.050488          -1.060808

In [15]:
visualize.get_bias_means(pos_df)

,,Association,Comp. association
Bias,Nationality,,
ryöstettävä,afro,1.328562,-0.029650
koulutettu,roma,1.020219,0.149509
puhdas,sami,1.003504,0.649674
vaalea,sami,0.655873,0.153662
tieteellinen,sami,0.590788,-1.674780
työllinen,roma,0.438451,-0.280055
kielitaidollinen,afro,0.385084,0.490215
huonompia,swe,0.255696,-0.304862
hyväksyvä,sami,0.103574,0.712458


In [16]:
means = visualize.get_bias_means(long_df)
# getting the biases which have a higher association for the ethnic group
means.loc[means['Association'] > means['Comp. association']]#.shape[0]

,,Association,Comp. association
Bias,Nationality,,
nössö,swe,6.616585,1.724446
terve,swe,5.579727,1.064115
oikeistolainen,swe,5.435727,-0.243132
ylipirteä,swe,5.344373,0.517601
ylimielinen,swe,5.219417,0.283808
rikas,swe,5.065080,0.392001
yläluokkainen,swe,4.575845,-0.064591
parempi,swe,4.087048,1.436999
tumma,sami,2.771914,2.095321


In [17]:
visualize.get_bias_means(long_pos_df)

,,Association,Comp. association
Bias,Nationality,,
sairas,swe,5.468915,0.934250
flegmaattinen,swe,5.431028,0.432793
rohkea,swe,5.095884,1.822477
köyhä,swe,5.010613,1.167036
nöyrä,swe,4.952120,0.596780
lempeä,swe,4.822954,0.925097
vasemmistolainen,swe,4.732029,0.110171
alaluokkainen,swe,4.705838,0.165303
vaalea,sami,3.192118,2.572613


In [18]:
# TODO
# make sure biases make sense/are translated correctly :)
# check if biased words in vocab D:

In [19]:
# make sure masking works correctly aksldjslfk eg venäläinen/suomalainen/romani(taustainen) vs suomenruotsalainen/saamelainen/somalialainen
# Use some other way of refering to ethnic group
# Romaninainen (Roma woman), Somalinainen (Somali woman), Somaliryhmä etc vs Kantasuomalainen, mies, etc
tokenizer.convert_tokens_to_ids("romanitaustainen") #suomalaisnainen etc
tokenized_text = tokenizer.tokenize("suomenruotsalaismies")
tokenized_text

['suomenruotsa', '##lais', '##mies']